### Read data

In [26]:
import json
from PIL import Image
from sklearn.model_selection import train_test_split    

def load_data_from_file(json_file_path, image_fold_path):
    with open(json_file_path, 'r') as f: #Open file to read file json
        data = json.load(f)

    dataset = []

    for entry_id, entry in data.items():
        img = Image.open(f'./{image_fold_path}/{entry['image']}')
        caption = entry['caption']
        if entry['label'] == 'not-sarcasm':
            label = 0
        elif entry['label'] == 'text-sarcasm':
            label = 1
        elif entry['label'] == 'image-sarcasm':
            label = 2
        else:
            label = 3
        dataset.append((entry_id, caption, label))
    return dataset

dataset = load_data_from_file('vimmsd-warmup.json', 'warmup-images')
print(dataset[:5])

# Chia dữ liệu với tỷ lệ 50 50
train_data, test_data = train_test_split(dataset, test_size=0.5, random_state=42)

print(f'Train size: {len(train_data)}, Test size: {len(test_data)}')


[('464', 'Biển miền Trung nước đẹp nhỉ', 3), ('7413', 'Chắc là nắc cụt rồi\n#phetphaikhong', 3), ('3808', 'Nhiều khi ta muốn ta được thiếu nợ\nĐể khi đi trốn có người đi tìmm', 0), ('5816', 'Phi công này 1 người lái thôi, ai đụng vào là chớt với chuỵ', 3), ('1632', 'Ủy ban Nhân dân thành phố Đà Nẵng vừa có văn bản về việc sẽ không tổ chức phun lửa, phun nước cầu Rồng và không quay cầu sông Hàn trong các đêm trình diễn Lễ hội pháo hoa Quốc tế Đà Nẵng (DIFF) 2023 để nhằm đảm bảo an toàn, hạn chế ùn tắc giao thông tại các cây cầu qua sông và khu vực trung tâm thành phố.\nCụ thể, không phun nước, phun lửa cầu Rồng trong các đêm 2/6, 10/6, 17/6, 24/6 và 8/7/2023. Không quay cầu sông Hàn trong các đêm 10/6, 17/6, 24/6 và 8/7/2023.', 0)]
Train size: 50, Test size: 51


### PREPARE DATA FOR PHOBERT

In [27]:
import torch
from torch.utils.data import Dataset, DataLoader #Dataset để định nghĩa tập dữ liệu, #DataLoader tạo các batch để nạp dữ liệu
from transformers import AutoTokenizer #Tải mô hình ngôn ngữ PhoBert

tokenizer = AutoTokenizer.from_pretrained('vinai/phobert-base', use_fast = False) 
# biến đổi câu thành token, ko dùng use_fast để đảm bảo có thể dùng phiên bản mặc định

class SarcasmTextDataset(Dataset): #Class dùng để quản lí dữ liệu cho việc train 
    def __init__(self, data, tokenizer, max_len=128):
        self.data = data #data bao gom caption va label
        self.tokenizer = tokenizer #doi van ban thanh token
        self.max_len = max_len #do dai toi da cua cau, dai hon se bi cat, ngan hon duoc dem them
    
    def __len__(self):
        return len(self.data) #tra ve do dai tap du lieu, so mau du lieu co trong tap du lieu

    def __getitem__(self, idx):
        entry_id, caption, label = self.data[idx] # lay caption va label cua data

        #tokenize
        encoding = self.tokenizer(
            caption,
            add_special_tokens = True, # them token dac biet [CLS], [SEP]
            max_length = self.max_len, # do dai toi da cua chuoi
            padding = 'max_length', # them padding neu chuoi ngan hon maxlen
            truncation = True, # cat bo chuoi qua dai
            return_tensors = 'pt', # tra ve du lieu dang pytorch
            clean_up_tokenization_spaces=False  # tắt việc dọn dẹp khoảng trắng trong token
        )
        return {
            'id': entry_id,  # Thêm id vào output
            'input_ids':encoding['input_ids'].squeeze(),
            'attention_mask':encoding['attention_mask'].squeeze(),
            'label': torch.tensor(label, dtype=torch.long)
        }
    
# Khởi tạo Dataset và DataLoader cho tập huấn luyện và kiểm tra
sarcasm_train = SarcasmTextDataset(train_data, tokenizer)
train_loader = DataLoader(sarcasm_train, batch_size=16, shuffle=True)

sarcasm_test = SarcasmTextDataset(test_data, tokenizer)
test_loader = DataLoader(sarcasm_test, batch_size=16, shuffle=False)


### FINE TUNE

In [28]:
import torch.nn as nn # thư viện chứa các thành phần cho học sâu, các hàm mất mát
from transformers import AutoModelForSequenceClassification # tải PhoBert cho nhiệm vụ phân loại
from torch.optim import AdamW # Bộ tối ưu hóa, điều chỉnh trọng số tránh overfitting

model = AutoModelForSequenceClassification.from_pretrained('vinai/phobert-base', num_labels = 4) # thêm label
model.to('cuda' if torch.cuda.is_available() else 'cpu') # dùng GPU nếu có hoặc CPU

# Set up tối ưu hóa
optimizer = AdamW(model.parameters(), lr=2e-5) # learning rate là 2e-5

# Set up hàm mất mát
criterion = nn.CrossEntropyLoss() # tính toán sự khác biệt label dự đoán và label thực tế

# Training
epochs = 3
device = 'cuda' if torch.cuda.is_available() else 'cpu'

for epoch in range(epochs):
    model.train() # dat model che do huan luyen
    total_loss = 0

    for batch in train_loader:
        # Move input to device
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        # zero gradients
        optimizer.zero_grad() # xóa các gradient ở các lần tính toán trước để tránh tích lũy

        # forward pass
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # backward pass
        loss.backward()

        # update weights
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f'Epoch {epoch + 1}, Loss: {avg_loss:.4f}')

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Keyword arguments {'clean_up_tokenization_spaces': False} not recognized.
Keyword arguments {'clean_up_tokenization_spaces': False} not recognized.
Keyword arguments {'clean_up_tokenization_spaces': False} not recognized.
Keyword arguments {'clean_up_tokenization_spaces': False} not recognized.
Keyword arguments {'clean_up_tokenization_spaces': False} not recognized.
Keyword arguments {'clean_up_tokenization_spaces': False} not recognized.
Keyword arguments {'clean_up_tokenization_spaces': False} not recognized.
Keyword arguments {'clean_up_tokenization_spaces': False} not recognized.
Keyword arguments {'clean_up_tokenization

Epoch 1, Loss: 1.3211


Keyword arguments {'clean_up_tokenization_spaces': False} not recognized.
Keyword arguments {'clean_up_tokenization_spaces': False} not recognized.
Keyword arguments {'clean_up_tokenization_spaces': False} not recognized.
Keyword arguments {'clean_up_tokenization_spaces': False} not recognized.
Keyword arguments {'clean_up_tokenization_spaces': False} not recognized.
Keyword arguments {'clean_up_tokenization_spaces': False} not recognized.
Keyword arguments {'clean_up_tokenization_spaces': False} not recognized.
Keyword arguments {'clean_up_tokenization_spaces': False} not recognized.
Keyword arguments {'clean_up_tokenization_spaces': False} not recognized.
Keyword arguments {'clean_up_tokenization_spaces': False} not recognized.
Keyword arguments {'clean_up_tokenization_spaces': False} not recognized.
Keyword arguments {'clean_up_tokenization_spaces': False} not recognized.
Keyword arguments {'clean_up_tokenization_spaces': False} not recognized.
Keyword arguments {'clean_up_tokenizat

Epoch 2, Loss: 1.0048


Keyword arguments {'clean_up_tokenization_spaces': False} not recognized.
Keyword arguments {'clean_up_tokenization_spaces': False} not recognized.
Keyword arguments {'clean_up_tokenization_spaces': False} not recognized.
Keyword arguments {'clean_up_tokenization_spaces': False} not recognized.
Keyword arguments {'clean_up_tokenization_spaces': False} not recognized.
Keyword arguments {'clean_up_tokenization_spaces': False} not recognized.
Keyword arguments {'clean_up_tokenization_spaces': False} not recognized.
Keyword arguments {'clean_up_tokenization_spaces': False} not recognized.
Keyword arguments {'clean_up_tokenization_spaces': False} not recognized.
Keyword arguments {'clean_up_tokenization_spaces': False} not recognized.
Keyword arguments {'clean_up_tokenization_spaces': False} not recognized.
Keyword arguments {'clean_up_tokenization_spaces': False} not recognized.
Keyword arguments {'clean_up_tokenization_spaces': False} not recognized.
Keyword arguments {'clean_up_tokenizat

Epoch 3, Loss: 1.1200


### evaluate

In [29]:
import json

def save_results_to_json(results, phase, output_file):
    output_data = {
        "results": results,
        "phase": phase
    }

    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(output_data, f, ensure_ascii=False, indent=4)
    print(f'Results saved to {output_file}')


def evaluate(model, dataloader, phase):
    model.eval()  # Đặt mô hình ở chế độ đánh giá
    correct = 0
    total = 0
    results = {}  # Khởi tạo dictionary để lưu kết quả dự đoán

    with torch.no_grad():  # Tắt tính toán gradient
        for idx, batch in enumerate(dataloader):
            # Di chuyển dữ liệu vào thiết bị
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            # Dự đoán
            outputs = model(input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs.logits, dim=1)  # Lấy chỉ số lớp có xác suất cao nhất

            # Cộng dồn số dự đoán đúng
            correct += (preds == labels).sum().item()
            total += labels.size(0)  # Cộng dồn tổng số mẫu

            # Lưu kết quả dự đoán vào dictionary với ID
            for i in range(len(preds)):
                sample_id = batch['id'][i]  # Lấy id từ batch
                if preds[i].item() == 0:
                    results[sample_id] = 'non-sarcasm'
                elif preds[i].item() == 1:
                    results[sample_id] = 'text-sarcasm'
                elif preds[i].item() == 2:
                    results[sample_id] = 'image-sarcasm'
                else:
                    results[sample_id] = 'multi-sarcasm'

    accuracy = correct / total if total > 0 else 0  # Tính độ chính xác
    print(f'Accuracy: {accuracy:.4f}')  # In ra độ chính xác

    # Lưu kết quả vào file JSON
    save_results_to_json(results, phase, 'results.json')




evaluate(model, test_loader, phase="warmup")  # Gọi hàm evaluate để đánh giá mô hình và lưu kết quả


Keyword arguments {'clean_up_tokenization_spaces': False} not recognized.
Keyword arguments {'clean_up_tokenization_spaces': False} not recognized.
Keyword arguments {'clean_up_tokenization_spaces': False} not recognized.
Keyword arguments {'clean_up_tokenization_spaces': False} not recognized.
Keyword arguments {'clean_up_tokenization_spaces': False} not recognized.
Keyword arguments {'clean_up_tokenization_spaces': False} not recognized.
Keyword arguments {'clean_up_tokenization_spaces': False} not recognized.
Keyword arguments {'clean_up_tokenization_spaces': False} not recognized.
Keyword arguments {'clean_up_tokenization_spaces': False} not recognized.
Keyword arguments {'clean_up_tokenization_spaces': False} not recognized.
Keyword arguments {'clean_up_tokenization_spaces': False} not recognized.
Keyword arguments {'clean_up_tokenization_spaces': False} not recognized.
Keyword arguments {'clean_up_tokenization_spaces': False} not recognized.
Keyword arguments {'clean_up_tokenizat

Accuracy: 0.5098
Results saved to results.json
